In [10]:
from PIL import Image
import pandas as pd
import json
import os
from sklearn.model_selection import train_test_split

seed = 42

In [12]:
#cargando json con el encoding de labels a un diccionario
ruta_json='../dataset/labelsBirmania.json'
with open(ruta_json, 'r') as f:
      labels_dict = json.load(f)

In [13]:
# leyendo todos los csv con las anotaciones de las predicciones en un dataframe
carpeta_csv = '../raw/Birmania/'

lista_dfs = []

for archivo in os.listdir(carpeta_csv):
    if archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta_csv, archivo)
        df = pd.read_csv(ruta_archivo)
        nombre_base = os.path.splitext(archivo)[0]
        df['nombre_carpeta'] = nombre_base
        df['frame'] = df['frame_id'].apply(lambda x: f'{x:02d}.jpg')
        lista_dfs.append(df)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(lista_dfs, ignore_index=True)
df_final.head()

,track_id,frame_id,x,y,w,h,label,nombre_carpeta,frame
0,_x4f29hsem,1,843,637,163,220,DNoHelmetP1NoHelmetP2NoHelmet,Bago_highway_1,01.jpg
1,_x4f29hsem,2,771,641,173,231,DNoHelmetP1NoHelmetP2NoHelmet,Bago_highway_1,02.jpg
2,_x4f29hsem,3,698,646,185,240,DNoHelmetP1NoHelmetP2NoHelmet,Bago_highway_1,03.jpg
3,_x4f29hsem,4,612,647,195,251,DNoHelmetP1NoHelmetP2NoHelmet,Bago_highway_1,04.jpg
4,_x4f29hsem,5,525,649,206,261,DNoHelmetP1NoHelmetP2NoHelmet,Bago_highway_1,05.jpg


In [14]:
# generando un dataframe que contenga un registro unico por imagen para
# particionar en train y test y luego parsear a formato yolov8
df_imagenes_unicas = df_final[['nombre_carpeta', 'frame']].drop_duplicates().reset_index(drop=True)
print(f"Imagenes unicas: {len(df_imagenes_unicas)}")

indices_imagenes = df_imagenes_unicas.index

train_indices, test_indices = train_test_split(
    indices_imagenes,
    test_size=0.3,
    random_state=seed
)

df_imagenes_train = df_imagenes_unicas.loc[train_indices].reset_index(drop=True)
df_imagenes_test = df_imagenes_unicas.loc[test_indices].reset_index(drop=True)

print(f"Número de imágenes para entrenamiento: {len(df_imagenes_train)}")
print(f"Número de imágenes para prueba: {len(df_imagenes_test)}")


Imagenes unicas: 100
Número de imágenes para entrenamiento: 70
Número de imágenes para prueba: 30


In [15]:
# funcion que dado un df de imagenes y carpeta genera tanto las imagenes como
# anotaciones en formato yolov8 en la carpeta dataset:

def create_dataset_yolo_from_raw(ruta_origen,ruta_destino_imagenes,ruta_destino_anotaciones,df_imagenes,df_anotaciones,image_width,image_height,target_size,labels_dict):
  for index, row in df_imagenes.iterrows():
    #copiando imagen
    nombre_carpeta = row['nombre_carpeta']
    frame_jpg = row['frame']
    frame = int(row['frame'].split('.')[0])
    ruta_imagen_origen = os.path.join(ruta_origen, nombre_carpeta, frame_jpg)
    ruta_imagen_destino = os.path.join(ruta_destino_imagenes, f'{nombre_carpeta.lower()}_{frame:03d}.jpg')
    #shutil.copy(ruta_imagen_origen, ruta_imagen_destino)

    try:
            img = Image.open(ruta_imagen_origen)
            img_resized = img.resize(target_size)
            img_resized.save(ruta_imagen_destino)
    except FileNotFoundError:
        print(f"Warning: Image not found at {ruta_imagen_origen}. Skipping.")
        continue # Saltar a la siguiente imagen si no se encuentra

    #generando anotaciones
    txt_filename = f"{nombre_carpeta.lower()}_{frame:03d}.txt"
    txt_filepath = os.path.join(ruta_destino_anotaciones, txt_filename)
    for index, row in df_anotaciones[(df_anotaciones.nombre_carpeta==nombre_carpeta) & (df_anotaciones.frame==frame_jpg)].iterrows():
      class_id=labels_dict[row['label']]
      x_center = int(row['x']+row['w']/2)
      y_center = int(row['y']+row['h']/2)

      x_center_norm = x_center / image_width
      y_center_norm = y_center / image_height
      width_norm = row['w'] / image_width
      height_norm = row['h'] / image_height

      yolov8_annotation=f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}\n"

      with open(txt_filepath, 'a') as f:
          f.write(yolov8_annotation)


In [18]:
#generando dataset de entrenamiento
create_dataset_yolo_from_raw(
    ruta_origen='../raw/Birmania',
    ruta_destino_imagenes='../dataset/birmania/images/train',
    ruta_destino_anotaciones='../dataset/birmania/labels/train',
    df_imagenes=df_imagenes_train,
    df_anotaciones=df_final,
    image_width=1920,
    image_height=1080,
    target_size=(640, 640),
    labels_dict=labels_dict,
)

In [19]:
#generando dataset de entrenamiento
create_dataset_yolo_from_raw(
    ruta_origen='../raw/Birmania',
    ruta_destino_imagenes='../dataset/birmania/images/val',
    ruta_destino_anotaciones='../dataset/birmania/labels/val',
    df_imagenes=df_imagenes_test,
    df_anotaciones=df_final,
    image_width=1920,
    image_height=1080,
    target_size=(640, 640),
    labels_dict=labels_dict,
)